# Week 3: representational similarity analysis (RSA)

This week's tutorial is about RSA! We'll be looking at how to transform patterns into RDMs using various distance measures, how to test the relation between feature-RDMs and brain-RDMs, and take a look at exploratory RDM visualization using multidimensional-scaling (MDS).

We will use data from the SharedStates dataset - a within-subject dataset used previously for a cross-decoding analysis (see [here](https://github.com/lukassnoek/SharedStates/blob/master/sharedstates_fullarticle_draft.pdf) for a draft of the corresponding article, which is currently in press). In the first couple of examples of this tutorial, we'll use the single-trial pattern estimates from the "self-task", in which subjects were shown short sentences about either emotional actions ("action" trials), emotional ("interoceptive") feelings ("interoception" trials), or emotional situations ("situation" trials). They were instructed to imagine as if they were experiencing/doing the actions/feelings/situations themselves (see figure below). <img src='self_task.png'>

The self-task was done twice (i.e. in two runs). Each run contained 20 trials of each condition (action, interoception, situation), so in total (across runs) we have 120 trials (20 trials \* 3 conditions \* 2 runs). While we applied a (cross-)decoding analysis on this dataset for the original study, we will apply some RSA techniques on this data for this week's tutorial. (The SharedStates dataset is, by the way, also one of the data-sets that you can use for your final project (more info on Blackboard/Week 4)).

In terms of skills, after this tutorial you are be able to:

* Create representational dissimilarity matrices (RDMs) from brain patterns using various metrics;
* Create custom "conceptual" feature-RDMs based on categorical conditions;
* Statistically test the similarity between feature- and brain-RDMS;
* Evaluate the explanatory power of your model relative to the data's noise using the *noise ceiling*;
* Create and test "computational" RDMs based on computationally-derived features;
* Exploratively visualize brain-RDMs with [MDS](http://scikit-learn.org/stable/auto_examples/manifold/plot_mds.html);

### Names
student 1: fill in your name ...

student 2: ... and the name of the person you're working with

In [ ]:
# Some imports for the rest of the tutorial
import os.path as op
from glob import glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Loading in and organizing data (yet again ...)
The SharedStates dataset is stored in the `/home/Public` folder again. Check out the folder (`/home/Public/SharedStates`) and its subfolders. You should see two directories - "SELF" and "OTHER" - which refer to the two tasks of the experiment. For the first part of the tutorial, you will work with data from the SELF task. Check out this directory: it should list several subdirectories referring to data from different subjects. Each subject contains two feat-directories referring to the two runs of the self-task. 

Let's check out the data from run 1 (`self1.feat`) for subject 2 (`sub002`). Look at the `design.png` file and make sure you understand the single-trial design. The `stats` directory contains the `tstat`-files corresponding to the single-trial-against-baseline contrasts. We want to load in these 60 files (\* 2, because there are two runs). 

<div class='alert alert-warning'>
**ToDo**: use `glob` to find *all* the tstat-paths belonging to *both* the runs from subject 2. So, you should find 120 paths. Hint: you need two wildcards.
</div>

In [ ]:
paths = 

In [ ]:
# Run this cell once you found the 120 paths (and stored it in the paths variable)
# This sorts the paths so that it is ordered *tstat1.nii.gz (run1), *tstat1.nii.gz (run2), *tstat2.nii.gz (run1) etc
paths = sorted(paths, key=lambda x: int(op.basename(x).split('.')[0].split('tstat')[-1]))

Below, we copied the Mvp-class from last week (we took the one we put in the solutions from last week, but you may also copy your own if you want!). 

In [ ]:
class Mvp():
    """ Custom class to load, organize, and process multivoxel MRI patterns. """
    
    def __init__(self, paths):
        
        self.paths = paths
        
    def load(self, voxel_dims=(91, 109, 91)):
        
        X = np.zeros((len(self.paths), np.prod(voxel_dims)))

        for i, path in enumerate(self.paths):
    
            X[i, :] = nib.load(path).get_data().ravel()
        
        self.X = X
    
    def standardize(self):
        self.X = (self.X - self.X.mean(axis=0)) / self.X.std(axis=0)
        # Because the above line may divide by 0 for voxels outside the brain,
        # this leads to NaN (Not a Number) values. The line below sets all NaN values to 0.
        self.X[np.isnan(self.X)] = 0
        
    def apply_mask(self, path_to_mask, threshold):
        
        mask = nib.load(path_to_mask).get_data()
        mask_bool = mask > threshold
        self.X = self.X[:, mask_bool.ravel()]

<div class='alert alert-warning'>
**ToDo**: initialize an Mvp-object below with the paths you globbed in the previous ToDo. Name the object `sub002`. Then, load the paths (call the method load()). Note that the voxel-dimensions are 80\*80\*37 here (i.e. in native EPI space, not standard MNI space); we included the voxel-dimensions as an argument for the `load()` function, so use it accordingly. Do **not** standardize it (we'll explain why in a bit). *Do* apply a mask (using the `apply_mask` method) using the `Supramarginal_Gyrus_Posterior` nifti-file that is included in the feat-directories of subject 2 (you can pick either of the two, they're the same). Use a threshold of 0. <br><br>

Check to see if everything worked: the `sub002.X` attribute should have a shape (samples-by-features) of 120\*2748 after applying the mask.
</div>

In [ ]:
# initialize the Mvp-object (and name it sub002)


### A note about standardization
Last week, you learned that you *need* to standardize your patterns (i.e. subtract the mean across samples and divide by the standard deviation across samples for all features (i.e. voxels) when we're working with ML algorithms. 

For RSA, however, you **shouldn't** standardize your patterns! It turns out that standardizing features leads to spurious (often negative) correlations between samples or conditions, which is of course detrimental for RSA. The reason for this is beyond the scope of this course (but see [this article](http://journal.frontiersin.org/article/10.3389/fnins.2013.00174/full) if you want to understand why this happens), but we thought you should know! 

## 2. Creating a 'conceptual model' RDM
In this section, we'll create a very simple 'conceptual model' RDM using the three conditions (action, interoception, situation) from the self-task. This conceptual model corresponds to the hypothesis that the distance between trials of the same condition are 0 and the distance between trials of different conditions is 1. If organized in a representational dissimilarity matrix (RDM), this would look like: <img src='conceptual_model.png'>

Importantly, we do not really use "world features" here, but we specify the RDM directly by *assuming* the dissimilarities between the samples (therefore, this type of model is often called a "conceptual" or "categorical" RDM). So, we're not "creating" a (feature-)RDM based on features, but based on an assumed (hypothesized) geometry. Later in the tutorial, we'll also test a computational model in which we *do* need to build a feature-RDM based on "world features" (instead of assuming a specific geometry).

But to create such a conceptual model RDM, we need to know to which condition (action, interoception, situation) each sample (trial) in our `Mvp` object belongs. One way is to manually write it down, given that you know which tstat-file belongs to which condition, like this:

`conditions = ['action', 'action', 'interoception', 'interoception', 'situation', 'situation', etc]`

If you have to do this for every subject, though, it becomes annoying and error-prone! Fortunately, there is also a "programmatic" way to do this. FSL in fact outputs a file called `design.con` when creating a `feat`-directory which lists the names you gave to the single-trial-against-baseline contrasts.

<div class='alert alert-warning'>
    **ToDo**: Check out the `design.con` file in the feat-directory of run1 (`self1.feat`) for subject 2 (just double-click the file, this will open the file in a text-editor). 
</div>

Although weirdly structured, you can see that the `design.con` file lists all the contrast-names, i.e. `actie_1`, `actie_2`, `interoception_1`, etc. (I accidentally used `actie`, the Dutch word for `action`). We have extracted the conditions for all the contrasts (single-trial estimates) for both runs for you already, which we stored in a text-file called `sample_labels.txt`. Below, we load in this file (and convert the list with strings to a numpy-array with strings):

In [ ]:
labels = np.loadtxt('/home/Public/SharedStates/SELF/sub002/sample_labels.txt',
                    dtype=str)
labels = np.array(labels)
print(labels)

<div class='alert alert-warning'>
**ToDo**: Programming challenge! Given the vector with labels of length *N* (N = number of samples), can you make a conceptual model RDM of shape *N*\**N*, in which cells corresponding to the same condition should be 0 and cells corresponding to different conditions should be 1 (basically, just like the image above)? Try to write it in a function - `make_conceptual_rdm(labels)` - that takes one argument (`labels`, a vector of length *N*) and returns an *N*\**N* RDM-matrix (a numpy array).<br><br>

**Note**: Do not assume that the labels are ordered (like it is now)! It should also work for vectors like `['A', 'A', 'B', 'C', 'B', 'A', 'B', etc]`<br>
**Hint**: it involves (at least) one for-loop.<br>
**Note**: This is meant as a programming exercise. Don't spend too much time on it! If you can't figure it out, you can use the function we have written, which is contained in `functions.py` (type `from functions import make_conceptual_rdm` to import the function).
</div>

In [ ]:
# implement it here


If you want to test your function, run the cell below! It also visualizes your RDM and how the RDM should look like! Because your function should work for any sequence of conditions, we also test a "shuffled" version of the labels.

In [ ]:
your_rdm = make_conceptual_rdm(labels)

# First check whether it really contains numbers (0 for same, 1 for different conditions)
assert(your_rdm.dtype != bool)

solution_rdm = np.load('solution_rdm.npy')

plt.figure(figsize=(8, 8))

plt.subplot(2, 2, 1)
plt.title('Your RDM')
plt.imshow(your_rdm, cmap='seismic')
plt.axis('off')
plt.subplot(2, 2, 3)
plt.title('The correct RDM')
plt.imshow(solution_rdm, cmap='seismic')
plt.axis('off')

# Check whether it is truly the same as the correct RDM
assert(np.sum(your_rdm == solution_rdm) == your_rdm.size)
print('Your RDM is correct!')

lab_shuf = [['actie', 'interoception', 'situation'] for i in range(40)]
lab_shuf = np.array([item for sublist in lab_shuf for item in sublist])

your_rdm_shuffled = make_conceptual_rdm(lab_shuf)
plt.subplot(2, 2, 2)
plt.title('Your RDM (w/shuffled labels)')
plt.imshow(your_rdm_shuffled, cmap='seismic')
plt.axis('off')

solution_shuffled_rdm = np.load('solution_shuffled_rdm.npy')

plt.subplot(2, 2, 4)
plt.title('The correct RDM (w/shuffled labels)')
plt.imshow(solution_shuffled_rdm, cmap='seismic')
plt.axis('off')
plt.tight_layout()
plt.show()

## 3. Creating your brain-RDM
While you specify your conceptual RDM directly, brain-RDMs have to be constructed by calculating distances between samples (in terms of their patterns in K-dimensional space). Therefore, when constructing brain-RDMs, you need to decide which distance metric you want to use. There are *very many* possible metrics (like correlation-distance, euclidean distance, mahalanobis distance, cityblock distance, etc. etc.), each with its own specific way of calculating distances, but fortunately they often yield very similar RDMs. 

You'll use `scikit-learn` later to calculate *all* the pairwise distances in an efficient manner, but let's practice with manually calculating distances before we go on.  

<div class='alert alert-warning'>
**ToDo**: Below, we 'extracted' the first two samples from the `X` attribute of the Mvp of subject 2. Now, calculate the euclidean distance between the two samples. (If you implemented it correctly, the distance should be 109.26)<br><br>

Hint: the formula for the euclidean distance ($\delta$) between the two vectors $p$ and $q$ with $K$ values (features) is:<br><br>

\begin{align}
\delta_{euclidean} = \sqrt{\sum_{i=1}^{K}{(p_{i} - q_{i})^{2}}}
\end{align}<br>
</div>

In [ ]:
sample1 = sub002.X[0, :]
sample2 = sub002.X[1, :]


Practically, calculating pairwise differences (i.e. sample 1 vs. sample 2, sample 1 vs. sample 3, etc. etc.) is quite tricky to implement (efficiently), as it involves N\*(N-1) pairwise distance calculations. Fortunately, scikit-learn has a module that contains the function `pairwise_distances` ([link](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html#sklearn.metrics.pairwise.pairwise_distances)) that does exactly what we want: it takes a samples-by-feature matrix and transforms it into a samples-by-samples (i.e. N\*N) distance matrix, the *RDM*.


<div class='alert alert-warning'>
**ToDo**: Below, we import the pairwise_distances function. Check out the documentation (click the link in the above text-block). Now, in the code-block below, call the pairwise_distances function with `sub002.X` as input and metric set to 'euclidean'. Store the result in a variable named `brain_rdm`. Check the shape - it it indeed an $N\cdot N$ array?
</div>

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

# Make the brain-RDM using pairwise_distances


Now, we can visualize RDMs using matplotlib's `imshow()` function:

In [ ]:
plt.figure(figsize=(6, 6))
plt.title('Brain RDM')
plt.imshow(brain_rdm, cmap='seismic')
plt.colorbar(label='Euclidean distance', fraction=0.046, pad=0.04)
plt.axis('off')
plt.show()

<div class='alert alert-warning'>
**ToDo**: we just calculated and visualized an RDM based upon the Euclidean distance, but as discussed, there are many others. Below, we created a list with different metrics. Create a loop over these metrics, in which you create a new RDM using the corresponding metric and visualize it using `plt.imshow()` across 3 different subplots.  <br><br>

Note: the cityblock distance is just the summed *absolute* distance between points (not the square root of the summed *squared* distance as in the euclidean) and the correlation-distance is simply 1 minus the correlation between two samples.
</div>

In [ ]:
metrics = ['cityblock', 'euclidean', 'correlation']

plt.figure(figsize=(15, 15))

# Start your loop over metrics here (this may take a while!)
for i, metric in enumerate(metrics):
    plt.subplot(1, 3, i+1)
    rdm = pairwise_distances(sub002.X, metric=metric)
    plt.title(metric, fontsize=15)
    plt.imshow(rdm, cmap='seismic')
    plt.axis('off')

plt.show()

<div class='alert alert-info'>
**(Advanced!) ToThink**: In the 3 subplots above, you clearly see that the `correlation` RDM is qualitatively different (and probably better, given our conceptual model) than the others (`cityblock` and `euclidean`). There is one factor that (probably) drives this difference. Can you figure out what this factor is? <br><br>

Hint 1: check out the formulas for the correlation distance, cityblock-distance and the euclidean distance between two vectors ("patterns") $p$ and $q$ which are of length $K$ (the amount of features/voxels), and have means $\bar{p}$ and $\bar{q}$:<br><br>

\begin{align}
\delta_{euclidean} = \sqrt{\sum_{i=1}^{K}{(p_{i} - q_{i})^{2}}}
\end{align}<br>

\begin{align}
\delta_{cityblock} = \sum_{i=1}^{K}{abs(p_{i} - q_{i})}
\end{align}<br>

\begin{align}
\delta_{correlation} = 1-\frac{\sum_{i=1}^{K}((p_{i} - \bar{p})\cdot(q_{i} - \bar{q}))}{\sqrt{\sum_{i=1}^{K}{((p_{i} - \bar{p})^{2}\cdot(q_{i} - \bar{q})^{2})}}}
\end{align}

</div>

In fact, let's just use the correlation-distance for our RDM, since the previous plot shows a better resemblance to our model RDM. (*NEVER* do this in practice. Because this is "data peeking", p-hacking, or whatever you want to name the process that inflates statistics by basing analysis decisions on the data/results itself.)

## 4. Testing RDMs
Alright, so now we got two RDMs: the conceptual model RDM (often called feature-RDM, or candidate-RDM) and the brain-RDM. To test whether our model (the conceptual model RDM) significantly explains our brain-RDM, we only have to correlate them, as depicted below:

In [ ]:
brain_rdm = pairwise_distances(sub002.X, metric='correlation')

In [ ]:
plt.subplot(1, 2, 1)
conc_rdm = np.load('solution_rdm.npy')
plt.title('Correlate this ...')
plt.imshow(conc_rdm, cmap='seismic')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('... with this!')
plt.imshow(brain_rdm, cmap='seismic')
plt.axis('off')
plt.show()

Well, not yet. We first have to do one thing. You see, the RDMs are symmetric: the values above and below the diagonal are exactly the same. And this should indeed be the case, because the distance between A and B should be the same as the distance between B and A. Or, in pattern analysis terms: the correlation/euclidean/cosine distance between the pattern of sample 1 and pattern of sample 2 is the same as that distance between the pattern of sample 2 and the pattern of sample 1.

Anyway, if we'd use the entire (flattened!) RDMs, we'd "artifically" create twice as many datapoints (i.e. the pairwise dissimilarities) than there really are, which will inflate the (significance of the) correlation between the RDMs (because of increased degrees of freedom). So, instead of using all $N\cdot N$ pairwise differences from the RDM, we need to extract only the $N\cdot(N-1)/2$ pairwise dissimilarity values (thus excluding the diagonal!).

Fortunately, numpy has a function to extract the upper (or, equivalently, lower) "triangle" of a square matrix (like our RDM) called `np.triu_indices` (triu = triangle upper; equivalent to `np.tril_indices` = triangle lower). This function gives the indices for the lower traingle of any $N\cdot N$ square matrix. Additionally, by using these indices to index the RDM, we ravel (flatten) the lower-triangle-extracted matrix into a vector, which is obviously necessary when we want to compute the correlation between them (because we cannot correlate matrices!). 

<div class='alert alert-warning'>
**ToDo**: Use the function `np.triu_indices` to extract the upper triangle of both the conceptual model RDM and the brain-RDM (store them in the variables `conc_rdm_triu` and `brain_rdm_triu` respectively). Check the [documentation](https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.triu.html) on how the `np.triu_indices` works (note: the function refers to $N$ as $m$). Also, by default the function includes the diagonal (which we don't want to include!), so you might have to fiddle with the `k` parameter in the function ... Make sure that your newly created variables are of size $N\cdot (N-1)/2$.
</div>

In [ ]:
# use np.triu_indices to index both conc_rdm and brain_rdm
conc_rdm_triu = 
brain_rdm_triu = 

In [ ]:
# also, check if your new variables are indeed of size N*(N-1)/2


Alright, finally we can test the correlation between the two (flattened) RDMs! Given that we're using a conceptual model (distance same condition = 0, distance different condition = 1), this tests whether, on average, brain patterns corresponding to samples from the same condition are more similar than brain patterns corresponding to samples from different conditions. 

Anyway, there is still one decision we need to make: what type of correlation should we choose? We could straightforwardly pick the convential Pearson correlation, but then we'd have to assume that the relationship between the feature-RDM and the brain-RDM is linear. For this and other reasons (see the appendix in [this article](http://journal.frontiersin.org/article/10.3389/neuro.06.004.2008/full) for more info), usually non-parametric correlations like the Spearman rank correlation or Kendall's Tau ($a$ version) are preferred. Here, we'll use the Spearman correlation.

<div class='alert alert-warning'>
**ToDo**: below, we import the `spearmanr` function (see [doc](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html)). Calculate the spearman correlation between the feature-RDM (conceptual model RDM) and the brain-RDM.
</div>

In [ ]:
from scipy.stats import spearmanr

# calculate the correlation here


<div class='alert alert-info'>
**ToThink**: In the previous ToDo, you should have found that the correlation between the conceptual RDM and the brain-RDM was about .18, which was (very!) significant ($p < 1.0^{-53}$). This p-value is, however, not valid, because the data (i.e. the RDMs) violate an assumption of the (Spearman) correlation. Can you think of which assumption this could be? (Hint: it's a similar assumption our data violates that we encountered in statistical tests of between-subject accuracy estimates.). Do you also know a solution? That is, do you know an alternative way to assess significance of this correlation?
</div>

Alright, that were basically all the steps necessary for a (subject-wise) RSA pipline. In summary, we did the following:

- Defined a candidate (feature-) RDM based on categorical distinctions;
- Loaded the brain-data in a samples-by-features format;
- Calculated the brain-RDM based on pairwise correlation distances;
- Computed the correlation between the feature-RDM and the brain-RDM

Now, usually we do not have a single subject (here: sub002) in our dataset, but multiple subjects. In that case, we'd perform the RSA pipeline on each subject, yielding a vector of subject-wise correlations of feature/brain RDMs, which can then subsequently be tested against zero using a (non-parametric) one-sample t-test. And this is exactly what you'll do in the next assignment!

<div class='alert alert-danger'>
**Assignment 1** (5 points): For this assignment, you should perform the analysis in which we correlated the conceptual model RDM with the brain-RDM *for each subject*. So, *glob* all subjects paths (i.e. `/home/Public/SharedStates/SELF/sub002`, `/home/Public/SharedStates/SELF/sub003`, etc.) and loop over it. Within the loop, do the following:<br><br>

- glob the tstat-paths from *both* feat-directories, and sort them like we did previously (with the key=lambda x ...);<br>
- initialize an Mvp-object, and load the paths;<br>
- load the `sample_labels.txt` file and create the conceptual model RDM based on the condition-labels;<br>
- create a brain-RDM using the `pairwise_distances` function (use `metric='correlation'`);<br>
- extract the lower (or upper) triangle of each RDM;<br>
- store the computed (lower-triangle-extract) brain-RDM vector in the pre-allocated variable `brain_rdms` (which we initialized already for you in the code block below), because we need this data in the next part of the tutorial;<br>
- calculate the spearman correlation between the two resulting vectors and store the correlation-value (not the p-value);<br>
- then, when you have 19 correlation values (there are 19 subjects), use a one-sample non-parametric t-test - called the [Wilcoxon signed-rank test](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) - to test whether the average correlation value is significantly above chance. You can use the `wilcoxon` function from the `scipy.stats` package (you have to import it first, of course).<br><br>

**Hint**: you should find that the mean spearman correlation is 0.1138 and the Wilcoxon p-value is 0.000155.

</div>

In [ ]:
brain_rdms = np.zeros((120*(120-1)/2, 19)) # matrix of (N*(N-1)/2)-by-19 (19 subjects)

# Start your Assignment here (pre-allocation of variables and the loop itself)


In [ ]:
# import the wilcoxon function and calculate the across-subject t-test of the correlation values


## 5. The noise ceiling
Sweet! In the previous assignment we found out that our conceptual model explains a significant amount of variance of our brain RDM. But, significance aside, we should ask ourselves: is this a (reasonably) *good* model? Surely, a 0.05 average correlation ($\rho$) is not quite high, given that correlations are expressed in a range from -1 (which would indicate a *horrible* model) to 1 (which would indicate a *perfect* model). On the other hand, we can ask ourselves what range of correlations we can reasonably expect *given the noise in our data*. We know that fMRI data (and thus pattern estimates/correlations) are extremely noisy, so maybe we should adjust our expectations about what a reasonable average correlation of our model with the brain-RDM would be.

Turns out that there is indeed a way to estimate an "upper bound" of model performance given the noise in your data: the **noise ceiling**. The noise ceiling represents the highest correlation ($\rho$) of any model to the brain-RDM that we can expect given the noise in the data. Usually (but depending on what type of correlation you use), it is calculated as the average (spearman) correlation of each subject's brain-RDM with the average brain-RDM across subjects (see the image below). <img src='noise_ceiling.png'>

<div class='alert alert-danger'>
**Assignment 2** (2 points): Remember that we saved the subject-specific RDMs in the `brain_rdms` variable? With this data, you can now calculate the noise ceiling for this analysis. So, do the following:<br><br>

- calculate the average (upper/lower-triangle-extracted) RDM (average across subjects);<br>
- loop over subjects and correlate (using `spearmanr`) the subject-specific RDM to the average RDM;<br>
- finally, average across the subject-specific (spearman) correlations to the average RDM to find the noise ceiling!<br><br>

Hint: you should find a noise ceiling of 0.30<br>
Note: if you were not able to complete the last assignment, you can uncomment the first line in the following code-block, which loads the correct `brain_rdms` variable we created earlier.
</div>

In [ ]:
# brain_rdms = np.load('brain_rdms_assignment1.npy')

# Find the noise ceiling!


## 6. Testing computational models with RSA
In the previous section we tested conceptual ('categorical') RDMs, which is a way to investigate assumed categorical distinctions in the representational geometry of brain patterns. Instead of assuming (the existence of) specific features, we can also test RDMS of *computationally derived features*. In that case, you're "simulating" the brain process that converts information in the input space (usually voxels) to meaningful features, and subsequently you're testing whether these computationally derived features are able to explain brain patterns in an RSA analysis.

In this section, we will explore the explanatory power RDMs based on features representing simple image statistics, derived from a model proposed by Steven himself (for more info, check [this](http://jov.arvojournals.org/article.aspx?articleid=2193462)). In short, the distribution of local contrast can be estimated from local receptive field responses using two summary statistics: the contrast energy (CE) and the spatial coherence (SC). In *natural scenes*, CE and SC correlate strongly with parameters of a Weibull function fitted to the distribution of local contrast values. CE is an approximation of the distribution width, informing about the overall presence of edges in an image, while SC is an approximation of its shape, capturing the higher-order correlation between those edges. Together, these parameters give an indication of the complexity of an image (whether an image is simple - with clear figure-ground segmentation, or more complex - with strong fragmentation). In other words, if you fit a weibull distribution on the pixels (the input space!) of an image, Steven's model outputs two features (the feature space!)\*: spatial coherence (SC; reflected by the $\gamma$ parameter of the weibull distribution) and contrast energy (CE; $\beta$ parameter of the weibull distribution). Thus, Steven's Weibull model is a perfect example for an (computational) RSA analysis, in which we test his model in terms of how well its RDM can explain the RDM of brain patterns.

***
\* In fact, Steven's model is somewhat more complicated than just fitting a Weibul distribution on images, but that's beyond the scope of this course.


### 6.1. Testing the Weibull model on the SharedStates data
It would be difficult to test the Weibull model on the stimuli from the "Self" task from SharedStates, however, because the stimuli consisted of visually similar (but semantically different) images, namely white text of a black background. The stimuli from the "Other" task, however, provide a better sample to fit the Weibull model on. 

Basically, the Other-task showed images of people in negative social situations. For each image, subjects had to infer the emotional state of the person in the image. Specifically, the subjects were asked to either try to infer **how** the person in the image expressed his/her emotion (condition: action), **what** the person in the image was feeling in terms of bodily experiences (condition: interoception), or **why** the person in the image was feeling the way he/she did (condition: situation). The specific focus (action, interoception, situation) was cued before a block of 5 images (using the cues "HOW", "WHAT", "WHY", respectively). In total, there were 30 images, which were each shown 3 times (once in the "action" context, once in the "interoception" context, and once in the "situation" context).
See the figure below for a schematic representation of the paradigm. <img src='other_task.png'>

While the original study aimed to show informational overlap between *self*-experienced emotions and perception of emotions in *others* using a cross-decoding analysis (train on trials in the self-task and cross-validate on trials in the other-task), we are going to completely ignore the condition labels (action, interoception, situation) for our computational RSA analyis. Instead, we are going to check out the SC and CE values derived from the images using the Weibull model. But first, let's check out the data in the feat-directories of the "other" task. 

<div class='alert alert-warning'>
**ToDo**: check out the the feat-directory of subject 2 (`/home/Public/SharedStates/OTHER/sub002/other.feat`). Open the `design.png` file (just double-click it) - can you make sense of it? It's basically, again, modelling each trial as a separate regressor and subsequently calculating contrasts-against-baseline from it. Note that we also alloted one regressor for modelling the cues.<br><br>

Mini ToThink: what is the potential beneficial effect of modelling the cues? <br><br>

Also check out the `stats` directory. It should contain 90 tstat-files. They're numbered 1 to 91, but that's because the contrast-image for the cue-regressor was also automatically generated (which originally corresponded to `tstat31.nii.gz`), but we removed this for you already.
</div>

Now, the feat-directory also contains a numpy datafile named `grayscale_complexity_params.npy`. These contain the two features (SC and CE) for all the 90 images that were presented (sorted to match the tstat-files). Let's load this file for subject 2 and check out the dimensions:

In [ ]:
complexity_sub002 = np.load('/home/Public/SharedStates/OTHER/sub002/other.feat/grayscale_complexity_params.npy')
print("Shape of complexity array: %s" % (complexity_sub002.shape,))

Alright, that makes sense, right? For every image (90 in total), there are two features (SC and CE). Notice that this is in the same "format" as our representation of brain patterns: samples-by-(brain)-features. 

Anyway, let's visualize these two features in a colorcoded plot (red = high values, blue = low values), in which the rows represent the samples (width is irrelevant):

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(complexity_sub002[:, 0, np.newaxis], aspect='auto', cmap='seismic')
plt.title('Spatial coherence')
plt.ylabel('Samples')
plt.xticks([])

plt.subplot(1, 2, 2)
plt.title('Contrast energy')
plt.imshow(complexity_sub002[:, 1, np.newaxis], aspect='auto', cmap='seismic')
plt.xticks([])
plt.yticks([])
plt.colorbar(label='feature strength')
plt.show()

<div class='alert alert-warning'>
**ToDo**: Make an RDM of the complexity array using the `pairwise_distances` function. Set metric to `correlation`. Then, visualize it using matplotlib's `imshow()` (set the `cmap` parameter to 'seismic').
</div>

Now we only need a brain-RDM that we think represents these computed features (SC and CE). Below, we glob the tstats belonging to the other-task of subject 2 and subsequently initialize an Mvp-object, load the files, and apply a mask (with threshold 30) of early visual cortex (V1) - an area that could very likely represent these visual features.

In [ ]:
paths = glob('/home/Public/SharedStates/OTHER/sub002/other.feat/stats/tstat*.nii.gz')
paths = sorted(paths, key=lambda x: int(op.basename(x).split('.')[0].split('tstat')[-1]))
mvp = Mvp(paths=paths)
mvp.load(voxel_dims=(80, 80, 37))
mvp.apply_mask('/home/Public/SharedStates/OTHER/sub002/other.feat/V1_bilateral.nii.gz', threshold=30)
print("Shape of X: %s" % (mvp.X.shape,))

<div class='alert alert-warning'>
**ToDo**: Well, you should know the drill by know. Make an RDM from the `X` attribute of your Mvp-object (use metric='correlation'), extract the upper-triangle from both the feature-RDM (based on the complexity parameters) and the brain-RDM, and correlate them using the Spearman correlation. (If you calculated it correctly, the correlation is 0.1243)
</div>

In [ ]:
# Implement the ToDo here!


A correlation of 0.1243, not bad! Officially, we need to statistically test whether this correlation is stable across subjects to conclude whether or not the Weibull features really explain a significant amount of variance in RDMs of early visual cortex. If you have time left, you can do this as specified in the next (optional) ToDo:

<div class='alert alert-warning'>
**ToDo** (optional): if you have time left, calculate the correlation between each subject's feature-RDM and (V1) brain-RDM by looping over subjects. Each subject's specific (sorted) complexity params are included in the feat-directory. Use `metric='correlation'` for both the feature and brain-RDMs. Hint: the final average correlation should be about 0.023. 
</div>

In [ ]:
# Implement the ToDO here


## 7. Exploratory RSA
Now, for this tutorial's last part, we're going to play around with some dimensionality reduction tools from, as always, `scikit-learn`. Let's use, for example, scikit-learn's `MDS` class. Objects of this class behave similar to regular transformers, i.e., they need to be `fit` and subsequently `transform` the RDM to yield the 2-dimensional coordinates that we can use in the exploratory visualization. Then, we use those coordinates to plot the original images in 2-dimensional space. Let's go through this step by step, using the V1 brain-RDM from subject 2 (which we defined earlier) as an example.

In [ ]:
from sklearn.manifold import MDS
mds = MDS(dissimilarity='precomputed', random_state=1)

In the cell above, we imported the MDS-class and initialized an MDS-instance. Importantly, we set the argument `dissimilarity` to 'precomputed'. Normally, you can specify which distance metric should be used in computing the MDS dimensionality reduction, but since we did this already (using the `pairwise_distances` function on the `X` attribute), we set it to 'precomputed'.

Now, we fit and transform the brain_rdm to get the coordinates. For some reason, the MDS-class only has a method that does these two steps at once: `fit_transform()`. So the only thing you have to do is:

In [ ]:
coordinates = mds.fit_transform(brain_rdm)

Now, we could simply plot the coordinates themselves ...

In [ ]:
plt.title('MDS coordinates')
plt.scatter(coordinates[:, 0], coordinates[:, 1])
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.show()

... but this doesn't tell us very much, because we cannot see which point belongs to which image. Therefore, we created a function for you guys that plots the corresponding images at the points in the MDS-plot. What the function does is simply loading the image corresponding to the particular sample and plotting it (using imshow) at the specified coordinates. 

After running the cell below, check out the plot. Do you see any "geometric structure"? (I haven't really, to be honest. Although you see that the same images - each image is presented three times - *kind of* cluster together, which makes sense for representatations based on patterns in visual cortex.)

In [ ]:
from functions import plot_2D_similarity_space
plot_2D_similarity_space(coordinates=coordinates,
                         stim_seq='/home/Public/SharedStates/OTHER/sub002/other.feat/stim_sequence.txt')

## 8. Permutation testing for RSA
As you've seen, whether a feature-RDM explains significant variance across subjects can be tested simply by a (non-parametric) t-test of your vector with subject-specific correlations against 0 (which represents chance-level correlation). This method, however, assumes that you have enough subjects (i.e. your sample is large enough) such that your statistical test is powerful enough. If you have only a few subjects (let's say $<13$), or even just a single subject, you probably don't have enough power for a t-test. 

Given that we can't "trust" the p-value corresponding to the Spearman correlation because of the violation of independence between data-points (pairwise dissimilarities in RDMs are *not* independent)\*, should we just conclude that there is no way to assess whether your observed correlation is signficant when our power is low? 

Well, no! Fortunately, there is permutation testing! Do you rembember the concept from last week? Permutation testing simply simulates the null-distribution by randomly shuffling the samples, such that any computations/statistics thereafter are based on a random structure. 

So, in the context of RSA, permutation testing is indeed recommended when power is low (generally $<13$). Suppose that you've only data from 3 subjects. What you could do, then, is average the three brain-RDMs together and subsequently correlate this with the corresponding feature-RDM *that has been created based on a randomly shuffled samples-by-features matrix*. Do this a 1000 times (each time randomly shuffling the samples-by-features matrix again before calculating the feature-RDM), and you'll have a solid (permuted) null-distribution. Then, you can calculate the corresponding p-value the way you're familiar with: as the proportion of permuted scores (here: correlations) higher than your observed score. 

Okay, let's look at an example. We'll first load a (simulated) conceptual RDM, which represents trials in which one of 6 emotions (anger, disgust, fear, happiness, sadness, surprise) were induced using emotion-inducing images:

***
\* This was the answer from one of the first ToThinks!

In [ ]:
frdm = np.load('emotion_conceptual_rdm.npy')
plt.figure(figsize=(5, 5))
plt.title('Hypothesized RDM of 6 emotion conditions')
plt.imshow(frdm, cmap='seismic')
plt.show()

Now, we load the brain-RDM (based on patterns in the bilateral insula) corresponding to the pairwise distance between those 96 trials (16 per emotion):

In [ ]:
brdm = np.load('emotion_brain_rdm.npy')
plt.figure(figsize=(5, 5))
plt.title('Brain RDM of 6 emotion conditions')
plt.imshow(brdm, cmap='seismic', vmin=.5)
plt.show()

And let's check out the correlation:

In [ ]:
frdm_tri = frdm[np.triu_indices(96, k=1)]
brdm_tri = brdm[np.triu_indices(96, k=1)]
corr, pval = spearmanr(frdm_tri, brdm_tri)
print("Correlation of %.3f (p = %.3f)" % (corr, pval))

Sweet! A significant correlation! Oh no, we know we can't trust those p-values :(
Now it's time to implement that permutation test!


<div class='alert alert-danger'>
**Assignment 3** (2 points): write a permutation-loop in which you randomly shuffle the feature-RDM (i.e. the variable `frdm_tri`) every time using `np.random.shuffle()` and subsequently fit the shuffled feature-RDM with the (non-shuffled) brain-RDM (i.e the variable `brdm_tri`). Use 5000 permutations. After you've calculated 5000 (simulated) null-correlations, calculate the p-value of our observed correlation using the formula from last week.
</div>

In [ ]:
# Implement the assignment here!
